# Educational Spendings Effect on Home Valuations Across the United States of America


In [29]:
# import libraries 

import pandas as pd
import numpy as np
import statsmodels.api as sm

# set float format
pd.options.display.float_format = '{:.2f}'.format

# Read datasets

gdpDeflator = pd.read_csv('data\FRED_ipd.csv')

homeValuation = pd.read_csv('data\ZillowHousingValues_state-by-state.csv')

educationSpending = pd.read_csv('data\education-spending_state-by-state.csv')

#### Format gdpDeflator to be used homeValuations and educationSpending. This table will eventually be used to merge and perform gdp deflation calculations.

In [30]:
# Rename columns
gdpDeflator = gdpDeflator.rename(columns={'DATE': 'year', 'USAGDPDEFAISMEI': 'ipd'})

# Set date to correct variable type
gdpDeflator['year'] = pd.to_datetime(gdpDeflator['year']) 

# remove all data that does not fit data date range
gdpDeflator = gdpDeflator[gdpDeflator['year'] >= '2000-01-01']
gdpDeflator = gdpDeflator[gdpDeflator['year'] <= '2016-01-01']

# reset index to accommodate new datarange
gdpDeflator = gdpDeflator.reset_index(drop=True)

#### Adjust Base year from 2015 to 2016

In [31]:
# set base year
base_year = '2016-01-01'

# calculate conversion factor
conversion_factor = gdpDeflator.loc[gdpDeflator['year'] == base_year, 'ipd'].values[0] / 100

# adjust to base year of 2016
gdpDeflator['ipd'] = gdpDeflator['ipd'] / conversion_factor

#### Format educationSpending to be merged with gdpDeflator (for inflation calculations)

In [32]:
# convert wide form dataframe into long form
educationSpending = pd.melt(educationSpending, id_vars=['state'], var_name='year', value_name='USD')

# change date variable type
educationSpending['year'] = pd.to_datetime(educationSpending['year'])

# sort by state, then year for easier indexing
educationSpending = educationSpending.sort_values(['state', 'year'])

# remove all data that does not fit data date range
educationSpending = educationSpending[educationSpending['year'] >= '2000-01-01']
educationSpending = educationSpending[educationSpending['year'] <= '2016-01-01']

# reset index after sort
educationSpending = educationSpending.reset_index(drop=True)

# rename columns
educationSpending = educationSpending.rename(columns={'USD': 'EducationSpend'})

#### Format homeValuation Dataframe

In [33]:
# drop unneccessary columns
collumnToDrop = ['RegionID', 'SizeRank', 'RegionType', 'StateName']
homeValuation = homeValuation.drop(collumnToDrop, axis=1)
# Convert wide form into long form dataframe
homeValuation = pd.melt(homeValuation, id_vars=['RegionName'], var_name='Date', value_name='HomeValue')

# convert ['Date'] datatype to DateTime
homeValuation['Date'] = pd.to_datetime(homeValuation['Date']) + pd.tseries.offsets.MonthBegin(-1)

# sort data by state and date
homeValuation = homeValuation.sort_values(['RegionName', 'Date'])

# Remove all data that does is not the start of a year
homeValuation = homeValuation[homeValuation['Date'].dt.month == 1]

# remove all data that does not fit data date range
homeValuation = homeValuation[homeValuation['Date'] <= '2016-01-01']
homeValuation = homeValuation[homeValuation['Date'] >= '2000-01-01']

# rename columns (allow for simple use of left join later)
homeValuation = homeValuation.rename(columns={'Date': 'year', 'RegionName': 'state'})

# reset NaN
homeValuation.dropna(inplace=True)

# reset index
homeValuation = homeValuation.reset_index(drop=True)

#### Left join educationSpending and gdpDeflator, to be used in gdp deflation calculations

In [34]:
# Left join
education_gdpDeflator = pd.merge(educationSpending, gdpDeflator, on='year', how='left')

# define variable for base year value
BaseYear = gdpDeflator.loc[16, 'ipd']

# Make new column for adjusted values
education_gdpDeflator['adjusted_EducationSpend'] = 0

# Adjust value calculations
education_gdpDeflator['adjusted_EducationSpend'] = (education_gdpDeflator['EducationSpend'] / education_gdpDeflator['ipd']) * BaseYear 

,state,year,adjusted_EducationSpend
0,Alabama,2000-01-01,5430164.89
1,Alabama,2001-01-01,5487005.84
2,Alabama,2002-01-01,5576575.72
3,Alabama,2003-01-01,5669021.11
4,Alabama,2004-01-01,5715963.43
5,Alabama,2005-01-01,5951621.55
6,Alabama,2006-01-01,6421525.28
7,Alabama,2007-01-01,6967551.44
8,Alabama,2008-01-01,7471373.40
9,Alabama,2009-01-01,7376747.43


In [ ]:
# Format final dataset
collumn_Drop = ['ipd', 'EducationSpend']
educationSpending_final = education_gdpDeflator.drop(collumn_Drop, axis=1)

educationSpending_final.head(20)

In [35]:
# Left Join
homeValuation_gdpDeflator = pd.merge(homeValuation, gdpDeflator, on='year', how='left')

# Make new column for adjusted values
homeValuation_gdpDeflator['adjusted_homeValuation'] = 0

# Inflation value adjustment calculations
homeValuation_gdpDeflator['adjusted_homeValuation'] = (homeValuation_gdpDeflator['HomeValue'] / homeValuation_gdpDeflator['ipd']) * BaseYear 

homeValuation_gdpDeflator.head(20)


,state,year,HomeValue,ipd,adjusted_homeValuation
0,Alabama,2000-01-01,78531.17,73.79,106425.87
1,Alabama,2001-01-01,81573.36,75.45,108112.99
2,Alabama,2002-01-01,84302.41,76.63,110015.34
3,Alabama,2003-01-01,86450.96,78.14,110635.58
4,Alabama,2004-01-01,89191.83,80.24,111159.30
5,Alabama,2005-01-01,93471.35,82.75,112950.95
6,Alabama,2006-01-01,98724.01,85.31,115727.42
7,Alabama,2007-01-01,106129.51,87.61,121134.64
8,Alabama,2008-01-01,108095.79,89.29,121057.11
9,Alabama,2009-01-01,105677.82,89.87,117595.49


In [37]:
# Format final dataset
collumn_Drop = ['ipd', 'HomeValue']
homeValuation_final = homeValuation_gdpDeflator.drop(collumn_Drop, axis=1)

homeValuation_final.head(20)

,state,year,adjusted_homeValuation
0,Alabama,2000-01-01,106425.87
1,Alabama,2001-01-01,108112.99
2,Alabama,2002-01-01,110015.34
3,Alabama,2003-01-01,110635.58
4,Alabama,2004-01-01,111159.30
5,Alabama,2005-01-01,112950.95
6,Alabama,2006-01-01,115727.42
7,Alabama,2007-01-01,121134.64
8,Alabama,2008-01-01,121057.11
9,Alabama,2009-01-01,117595.49
